## Detail - Level 1

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='data',  train=True, 
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, 
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the MLP model
model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the model
max_epoch = 5
for epoch in range(max_epoch):
    for i, (inputs, labels) in enumerate(trainloader, 0):
        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{max_epoch}]")

Epoch [1/5]
Epoch [2/5]
Epoch [3/5]
Epoch [4/5]
Epoch [5/5]


In [2]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        # Move inputs and labels to the device
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"accuracy: {accuracy}")

accuracy: 83.98


## Detail - Level 2

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='data', train=True, 
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, 
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the MLP model
model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training the model
max_epoch = 5
for epoch in range(max_epoch):
    running_loss = 0.0
    correct = 0   # to track number of correct predictions
    total = 0     # to track total number of samples

    for i, (inputs, labels) in enumerate(trainloader, 0):
        # see comments from the previous example 
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # accumulate loss
        running_loss += loss.item()

    epoch_accuracy = 100 * correct / total
    running_loss = running_loss / (i + 1)
    
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

Epoch [1/5], Loss: 0.7962, Accuracy: 74.30%
Epoch [2/5], Loss: 0.5184, Accuracy: 81.81%
Epoch [3/5], Loss: 0.4681, Accuracy: 83.50%
Epoch [4/5], Loss: 0.4408, Accuracy: 84.45%
Epoch [5/5], Loss: 0.4204, Accuracy: 85.14%


## Detail - Level 3

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='data', train=True, 
                                             download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, 
                                            download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Define the MLP model
model = nn.Sequential(
    nn.Flatten(), 
    nn.Linear(784, 256),
    nn.ReLU(),
    nn.Linear(256, 10)
)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Function to compute loss and accuracy for test set
def evaluate(model, testloader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            # Move inputs and labels to the device
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return test_loss / len(testloader), accuracy

# Training the model
for epoch in range(5):
    running_loss = 0.0
    correct = 0   # to track number of correct predictions
    total = 0     # to track total number of samples

    for i, (inputs, labels) in enumerate(trainloader, 0):
        # see comments from the previous example 
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # Determine class predictions and track accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        # accumulate loss
        running_loss += loss.item()

    epoch_accuracy = 100 * correct / total
    running_loss = running_loss / (i + 1)    
    test_loss, test_accuracy = evaluate(model, 
                                        testloader, 
                                        criterion)
    
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {running_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

print('Finished Training')

Epoch [1/5], Loss: 0.7924, Accuracy: 74.46%, Test Loss: 0.5780, Test Accuracy: 79.37%
Epoch [2/5], Loss: 0.5182, Accuracy: 81.86%, Test Loss: 0.5101, Test Accuracy: 81.55%
Epoch [3/5], Loss: 0.4694, Accuracy: 83.55%, Test Loss: 0.4863, Test Accuracy: 82.28%
Epoch [4/5], Loss: 0.4418, Accuracy: 84.43%, Test Loss: 0.4604, Test Accuracy: 83.67%
Epoch [5/5], Loss: 0.4225, Accuracy: 85.14%, Test Loss: 0.4471, Test Accuracy: 83.87%
Finished Training
